Implementation and comparision of LSTM(Long Short Term Memory) and ARIMA(Autoregressive integrated moving average) Neural Networks for the application of Stock Price Prediction.

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

We shall follow the following steps to achieve the goal:
1. Fetch and store the data(Stock Prices for the past few time intervals).
2. Split the data for Training and Testing.
3. Processing of data to meet the input criterion of the Neural Network Algorithms.
4. Pass the data through LSTM and ARIMA and train the networks.
5. Start Predicting the upcoming Stock Prices for the next few time intervals using the 2 models.
6. Compare the results of the 2 models and conclude.

In [5]:
## Fetch and Store the data
#
# Function Description  : Fetches and Stores the required data into an array
#
# Output Parameters     : data(The Data used for training and prediction)       : data
##
def fetch_store ():
    length_data = 10000
    data = range(1, length_data)
    return data


## Split Data Function
#
# Function Description  : Splits data into 2 sets : Train Data, Test Data
#
# Input Parameters      : 1. data (The data that is to be splitted)                             : data
#                         2. percentage_train_data (The percentage of data used for training)   : percentage_train_data
#
# Output Parameters     : 1. Train Data (1st part of Data)                                      : train_data
#                         2. Test Data (2nd part of Data)                                       : test_data
##
def split_data (data, percentage_train_data):
    # Gets the length of the data
    data_length = len(data)

    # Gets the length of the Training data
    train_data_length = int (percentage_train_data*data_length)

    # Splits the data into 2 sets : Train data and Test data
    train_data = data[:train_data_length]
    test_data = data[train_data_length:]

    return train_data, test_data


## Preprocessing of Data
#
# Function Description  : Precprocesses the train and test data as per the crierion of the Neural Networks.
#
# Criterion             : Scale data to range [0, 1]
#
# Input Parameters      : 1. Train Data                 : train_data
#                         2. Test Data                  : test_data
#
# Output Parameters     : 1. Processed Train Data       : processed_train_data
#                         2. Processed Test Data        : processed_test_data
#                         3. Scaling Factor used        : scaling_factor
##
def preprocessing_data (train_data, test_data):
    # Finding maximum among the Train and Test Data
    max_train_data = max(train_data)
    max_test_data = max(test_data)

    scaling_factor = max(max_train_data, max_test_data)
    

    # Scaling the data to the range [0, 1]
    processed_train_data = train_data/scaling_factor
    processed_test_data = test_data/scaling_factor

    return processed_train_data, processed_test_data, scaling_factor

